In [0]:
pip install mlflow

Python interpreter will be restarted.
  Using cached mlflow-2.4.1-py3-none-any.whl (18.1 MB)
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
  Using cached docker-6.1.3-py3-none-any.whl (148 kB)
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached alembic-1.11.1-py3-none-any.whl (224 kB)
  Using cached Flask-2.3.2-py3-none-any.whl (96 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached importlib_metadata-6.7.0-py3-none-any.whl (22 kB)
  Using cached SQLAlchemy-2.0.17-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.8 MB)
  Using cached sqlparse-0.4.4-py3-none-any.whl (41 kB)
  Using cached databricks_cli-0.17.7-py3-none-any.whl
  Using cached Mako-1.2.4-py3-none-any.whl (

In [0]:
import mlflow
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import numpy as np
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error,mean_absolute_error

#Day50

In [0]:
TrainDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TrainDf50/')

TestDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TestDf50/')

x_train = TrainDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_train = TrainDf.select("M305305Vetor").\
    rdd.flatMap(lambda x: x).collect()

x_test = TestDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_test = TestDf.select("M305305Vetor").\
    rdd.flatMap(lambda x: x).collect()


featureList =['MultiparousCow','Autumn','Spring','Summer','Magnitude','TimeToPeakYield','Offset','Decay','TestDayMilkYield','AgeInMonths','HM305','MeanMagnitude', 'MeanTimeToPeakYield', 'MeanOffset', 'MeanDecay']

In [0]:
X = sm.add_constant(x_train)
mod = sm.OLS(y_train,X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     4104.
Date:                Sun, 02 Jul 2023   Prob (F-statistic):               0.00
Time:                        14:14:45   Log-Likelihood:            -1.4853e+05
No. Observations:               18074   AIC:                         2.971e+05
Df Residuals:                   18058   BIC:                         2.972e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.016e+04     47.293    214.751      0.0

## train

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    
    # model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
    coeffs = lm.coef_.tolist()[0]
    y_pred = lm.predict(x_train)
   
    # log parameters
    mlflow.log_param('data','train')
    mlflow.log_param('type','whole')
    mlflow.log_param('day','50')
    
    plt.figure(figsize=(20,7))
    plt.bar(featureList,coeffs)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.ylabel('β', fontsize=20,rotation=0)
    plt.xticks(rotation=25,fontsize= 13)
    plt.yticks(fontsize= 13)
    plt.savefig("β.png",bbox_inches='tight')
    plt.close()
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_train,y_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_train,y_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_train,y_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_train,y_pred))
    mlflow.log_artifact("β.png")

## test

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    # train the model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
       
    # use the model to predict the test data
    
    y_test_pred = lm.predict(x_test)
   
    # log parameters
    mlflow.log_param('data','test')
    mlflow.log_param('type','whole')
    mlflow.log_param('day','50')
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_test,y_test_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_test,y_test_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_test,y_test_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_test,y_test_pred))
    

#Day75

In [0]:
TrainDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TrainDf75/')

TestDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TestDf75/')

x_train = TrainDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_train = TrainDf.select("M305305Vetor").\
    rdd.flatMap(lambda x: x).collect()

x_test = TestDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_test = TestDf.select("M305305Vetor").\
    rdd.flatMap(lambda x: x).collect()


featureList =['MultiparousCow','Autumn','Spring','Summer','Magnitude','TimeToPeakYield','Offset','Decay','TestDayMilkYield','AgeInMonths','HM305','MeanMagnitude', 'MeanTimeToPeakYield', 'MeanOffset', 'MeanDecay']

In [0]:
X = sm.add_constant(x_train)
mod = sm.OLS(y_train,X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                     8324.
Date:                Sun, 02 Jul 2023   Prob (F-statistic):               0.00
Time:                        14:16:20   Log-Likelihood:            -1.7687e+05
No. Observations:               22006   AIC:                         3.538e+05
Df Residuals:                   21990   BIC:                         3.539e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.016e+04     35.248    288.329      0.0

## train

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    
    # model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
    coeffs = lm.coef_.tolist()[0]
    y_pred = lm.predict(x_train)
   
    # log parameters
    mlflow.log_param('data','train')
    mlflow.log_param('type','whole')
    mlflow.log_param('day','75')
    
    plt.figure(figsize=(20,7))
    plt.bar(featureList,coeffs)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.ylabel('β', fontsize=20,rotation=0)
    plt.xticks(rotation=25,fontsize= 13)
    plt.yticks(fontsize= 13)
    plt.savefig("β.png",bbox_inches='tight')
    plt.close()
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_train,y_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_train,y_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_train,y_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_train,y_pred))
    mlflow.log_artifact("β.png")

## test

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    # train the model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
       
    # use the model to predict the test data
    
    y_test_pred = lm.predict(x_test)
   
    # log parameters
    mlflow.log_param('data','test')
    mlflow.log_param('type','whole')
    mlflow.log_param('day','75')
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_test,y_test_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_test,y_test_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_test,y_test_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_test,y_test_pred))
    

#Day100

In [0]:
TrainDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TrainDf100/')

TestDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TestDf100/')

x_train = TrainDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_train = TrainDf.select("M305305Vetor").\
    rdd.flatMap(lambda x: x).collect()

x_test = TestDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_test = TestDf.select("M305305Vetor").\
    rdd.flatMap(lambda x: x).collect()


featureList =['MultiparousCow','Autumn','Spring','Summer','Magnitude','TimeToPeakYield','Offset','Decay','TestDayMilkYield','AgeInMonths','HM305','MeanMagnitude', 'MeanTimeToPeakYield', 'MeanOffset', 'MeanDecay']

In [0]:
X = sm.add_constant(x_train)
mod = sm.OLS(y_train,X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.893
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                 1.069e+04
Date:                Sun, 02 Jul 2023   Prob (F-statistic):               0.00
Time:                        14:17:02   Log-Likelihood:            -1.5107e+05
No. Observations:               19159   AIC:                         3.022e+05
Df Residuals:                   19143   BIC:                         3.023e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.026e+04     31.799    322.657      0.0

## train

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    
    # model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
    coeffs = lm.coef_.tolist()[0]
    y_pred = lm.predict(x_train)
   
    # log parameters
    mlflow.log_param('data','train')
    mlflow.log_param('type','whole')
    mlflow.log_param('day','100')
    
    plt.figure(figsize=(20,7))
    plt.bar(featureList,coeffs)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.ylabel('β', fontsize=20,rotation=0)
    plt.xticks(rotation=25,fontsize= 13)
    plt.yticks(fontsize= 13)
    plt.savefig("β.png",bbox_inches='tight')
    plt.close()
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_train,y_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_train,y_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_train,y_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_train,y_pred))
    mlflow.log_artifact("β.png")

## test

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    # train the model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
       
    # use the model to predict the test data
    
    y_test_pred = lm.predict(x_test)
   
    # log parameters
    mlflow.log_param('data','test')
    mlflow.log_param('type','whole')
    mlflow.log_param('day','100')
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_test,y_test_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_test,y_test_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_test,y_test_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_test,y_test_pred))
    

#Day125

In [0]:
TrainDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TrainDf125/')

TestDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TestDf125/')

x_train = TrainDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_train = TrainDf.select("M305305Vetor").\
    rdd.flatMap(lambda x: x).collect()

x_test = TestDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_test = TestDf.select("M305305Vetor").\
    rdd.flatMap(lambda x: x).collect()


featureList =['MultiparousCow','Autumn','Spring','Summer','Magnitude','TimeToPeakYield','Offset','Decay','TestDayMilkYield','AgeInMonths','HM305','MeanMagnitude', 'MeanTimeToPeakYield', 'MeanOffset', 'MeanDecay']

In [0]:
X = sm.add_constant(x_train)
mod = sm.OLS(y_train,X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                 1.186e+04
Date:                Sun, 02 Jul 2023   Prob (F-statistic):               0.00
Time:                        14:17:43   Log-Likelihood:            -1.1347e+05
No. Observations:               14701   AIC:                         2.270e+05
Df Residuals:                   14685   BIC:                         2.271e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.03e+04     31.342    328.616      0.0

## train

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    
    # model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
    coeffs = lm.coef_.tolist()[0]
    y_pred = lm.predict(x_train)
   
    # log parameters
    mlflow.log_param('data','train')
    mlflow.log_param('type','whole')
    mlflow.log_param('day','125')
    
    plt.figure(figsize=(20,7))
    plt.bar(featureList,coeffs)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.ylabel('β', fontsize=20,rotation=0)
    plt.xticks(rotation=25,fontsize= 13)
    plt.yticks(fontsize= 13)
    plt.savefig("β.png",bbox_inches='tight')
    plt.close()
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_train,y_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_train,y_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_train,y_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_train,y_pred))
    mlflow.log_artifact("β.png")

## test

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    # train the model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
       
    # use the model to predict the test data
    
    y_test_pred = lm.predict(x_test)
   
    # log parameters
    mlflow.log_param('data','test')
    mlflow.log_param('type','whole')
    mlflow.log_param('day','125')
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_test,y_test_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_test,y_test_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_test,y_test_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_test,y_test_pred))
    